## K26 - Heated Wall - Part 2: Postprocessing

Interface at 90°.  
Equal fluid densities
Also no Heat capacity => infinitely fast heat conduction  
Height of the domain is reduced  

#### Instructions

This worksheet serves as a basis to conduct various parameter studies for the Heated Wall setup.

### Step 1 - Initialization

Load the BoSSS code, do not change

In [ ]:
//#r "..\..\..\src\L4-application\BoSSSpad\bin\Release\net5.0\BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

### Step 2 - Workflowmanagement, Batchprocessor and Database

In [ ]:
ExecutionQueues

index,type,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,Username,ServerName,ComputeNodes,DefaultJobPriority,SingleNode,AllowedDatabasesPaths,BatchInstructionDir
0,BoSSS.Application.BoSSSpad.MsHPC2012Client,\\hpccluster\hpccluster-scratch\rieckmann\binaries,True,<null>,dotnet,FDY\rieckmann,DC2,<null>,Normal,True,"[ { \\hpccluster\hpccluster-scratch\rieckmann == : LocalMountPath: \\hpccluster\hpccluster-scratch\rieckmann, PathAtRemote: } ]",
1,BoSSS.Application.BoSSSpad.MsHPC2012Client,\\hpccluster\hpccluster-scratch\rieckmann\binaries,True,HPCCLUSTER,dotnet,FDY\rieckmann,DC2,[ hpccluster ],Normal,True,"[ { \\hpccluster\hpccluster-scratch\rieckmann\BoSSS_DB == : LocalMountPath: \\hpccluster\hpccluster-scratch\rieckmann\BoSSS_DB, PathAtRemote: } ]",
2,BoSSS.Application.BoSSSpad.MsHPC2012Client,\\hpccluster\hpccluster-scratch\rieckmann\binaries,True,HPCCLUSTER2,dotnet,FDY\rieckmann,DC2,[ hpccluster2 ],Normal,True,"[ { \\hpccluster\hpccluster-scratch\rieckmann\BoSSS_DB == : LocalMountPath: \\hpccluster\hpccluster-scratch\rieckmann\BoSSS_DB, PathAtRemote: } ]",
3,BoSSS.Application.BoSSSpad.MiniBatchProcessorClient,C:\Users\rieckmann\AppData\Local\BoSSS-LocalJobs,False,<null>,dotnet,,,,,,,<null>


In [ ]:
string ProjectName = $"HeatedWall_Validation";
BoSSSshell.WorkflowMgm.Init(ProjectName);

Project name is set to 'HeatedWall_Validation'.


In [ ]:
static var myBatch = BoSSSshell.GetDefaultQueue();

In [ ]:
static var myDb = BoSSSshell.WorkflowMgm.DefaultDatabase;

In [ ]:
myDb.Path

Opening existing database '\\hpccluster\hpccluster-scratch\rieckmann\XNSFE_HeatedWall'.


\\hpccluster\hpccluster-scratch\rieckmann\XNSFE_HeatedWall

In [ ]:
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();

### Step 3 - Load session data

#### Step 3.1 - Reference sessions

In [ ]:
using System.IO.Compression;

In [ ]:
ZipFile.ExtractToDirectory("./HeatedWall_Validation.zip", "./HeatedWall_Validation");

In [ ]:
var dbRef = BoSSSshell.OpenOrCreateDatabase(@"./HeatedWall_Validation");
var sessionsRef = dbRef.Sessions;
sessionsRef

Opening existing database './HeatedWall_Validation'.


#0: XNSFE_HeatedWall_Convergence	HeatedWall_res:256_p:3_dR:0_MCL:False	10/15/2021 10:42:49	3a5e2432...
#1: XNSFE_HeatedWall_Convergence	HeatedWall_res:256_p:2_dR:0_MCL:False	10/15/2021 10:42:32	38c9ca48...
#2: XNSFE_HeatedWall_Convergence	HeatedWall_res:128_p:3_dR:0_MCL:False	10/15/2021 10:40:24	95969c1b...
#3: XNSFE_HeatedWall_Convergence	HeatedWall_res:128_p:2_dR:0_MCL:False	10/15/2021 10:40:24	338be26b...
#4: XNSFE_HeatedWall_Convergence	HeatedWall_res:64_p:3_dR:0_MCL:False	10/15/2021 10:40:24	2cfb6071...
#5: XNSFE_HeatedWall_Convergence	HeatedWall_res:64_p:2_dR:0_MCL:False	10/15/2021 10:40:24	7568a07b...
#6: XNSFE_HeatedWall_Convergence	HeatedWall_res:32_p:2_dR:0_MCL:False	10/15/2021 10:40:24	a983548a...
#7: XNSFE_HeatedWall_Convergence	HeatedWall_res:32_p:3_dR:0_MCL:False	10/15/2021 10:40:24	0babfb57...
#8: XNSFE_HeatedWall_Convergence	HeatedWall_res:16_p:3_dR:0_MCL:False	10/15/2021 10:40:24	f768975f...
#9: XNSFE_HeatedWall_Convergence	HeatedWall_res:16_p:2_dR:0_MCL:False	10/15/20

Remove the reference database again, so the reference sessions don't get loaded twice.

In [ ]:
BoSSSshell.databases = BoSSSshell.databases.Where(db => db != dbRef).ToArray();

#### Step 3.2 - New sessions

In [ ]:
// load all successful sessions
var sessions = BoSSSshell.WorkflowMgm.Sessions.Where(s => s.SuccessfulTermination);
sessions

#0: HeatedWall_Validation	HeatedWall_Validation_GridRes_256_DgDegree_2_HeatFlux_0.02_DensityRatio_1	10/28/2021 18:05:28	e41ad1bb...
#1: HeatedWall_Validation	HeatedWall_Validation_GridRes_256_DgDegree_2_HeatFlux_0.002_DensityRatio_1	10/28/2021 18:02:27	07db5d00...
#2: HeatedWall_Validation	HeatedWall_Validation_GridRes_256_DgDegree_2_HeatFlux_0_DensityRatio_1	10/28/2021 17:18:29	887defe7...
#3: HeatedWall_Validation	HeatedWall_Validation_GridRes_128_DgDegree_2_HeatFlux_0.002_DensityRatio_1	10/28/2021 18:00:46	83e3ab69...
#4: HeatedWall_Validation	HeatedWall_Validation_GridRes_128_DgDegree_2_HeatFlux_0.02_DensityRatio_1	10/28/2021 18:03:51	0c5e08ba...
#5: HeatedWall_Validation	HeatedWall_Validation_GridRes_128_DgDegree_2_HeatFlux_0_DensityRatio_1	10/28/2021 17:16:24	686cf0af...
#6: HeatedWall_Validation	HeatedWall_Validation_GridRes_64_DgDegree_2_HeatFlux_0.02_DensityRatio_1	10/28/2021 18:03:15	7127f534...
#7: HeatedWall_Validation	HeatedWall_Validation_GridRes_64_DgDegree_2_HeatFlux_0.

In [ ]:
var allSessions = sessions.Cat(sessionsRef);

#### Step 3.3 - Create DataTables

In [ ]:
using System.Data;

In [ ]:
DataTable sessionTable = new DataTable("SessionTable");

DataColumn column;
// first column session id
column = new DataColumn();
column.DataType = typeof(Guid);
column.ColumnName = "SessionID";
column.ReadOnly = true;
column.Unique = true;
sessionTable.Columns.Add(column);
// second column study #
column = new DataColumn();
column.DataType = typeof(List<int>);
column.ColumnName = "Study#";
column.ReadOnly = false;
column.Unique = false;
sessionTable.Columns.Add(column);
// Make the Guid column the primary key column.
DataColumn[] PrimaryKeyColumns = new DataColumn[1];
PrimaryKeyColumns[0] = sessionTable.Columns["SessionID"];
sessionTable.PrimaryKey = PrimaryKeyColumns;

In [ ]:
DataTable studyTable = new DataTable("StudyTable");

DataColumn column;
// first column study #
column = new DataColumn();
column.DataType = typeof(int);
column.ColumnName = "Study#";
column.AutoIncrement = true;
column.ReadOnly = true;
column.Unique = true;
studyTable.Columns.Add(column);
// second column number of reference study
column = new DataColumn();
column.DataType = typeof(int);
column.ColumnName = "RefStudy#";
column.AutoIncrement = true;
column.ReadOnly = true;
column.Unique = false;
studyTable.Columns.Add(column);
// third column sessions
column = new DataColumn();
column.DataType = typeof(Guid[]);
column.ColumnName = "SessionIDs";
column.ReadOnly = true;
column.Unique = false;
studyTable.Columns.Add(column);
// fourth column description
column = new DataColumn();
column.DataType = typeof(string);
column.ColumnName = "Study Description";
column.ReadOnly = false;
column.Unique = false;
studyTable.Columns.Add(column);
// fifth column contact line log data
column = new DataColumn();
column.DataType = typeof(Plot2Ddata[]);
column.ColumnName = "Contactline Plot Data";
column.ReadOnly = false;
column.Unique = false;
studyTable.Columns.Add(column);
// sixth column massflux log data
column = new DataColumn();
column.DataType = typeof(Plot2Ddata[]);
column.ColumnName = "Massflux Plot Data";
column.ReadOnly = false;
column.Unique = false;
studyTable.Columns.Add(column);
// seventh column convergence data
column = new DataColumn();
column.DataType = typeof(Plot2Ddata[]);
column.ColumnName = "Convergence Data";
column.ReadOnly = false;
column.Unique = false;
studyTable.Columns.Add(column);
// Make the study # column the primary key column.
DataColumn[] PrimaryKeyColumns = new DataColumn[1];
PrimaryKeyColumns[0] = studyTable.Columns["Study#"];
studyTable.PrimaryKey = PrimaryKeyColumns;

#### Step 3.4 - Fill sessionTable

In [ ]:
sessionsRef.ForEach(s => {
    var row = sessionTable.NewRow();
    if(sessionTable.Rows.Find(s.ID) == null){
        row["SessionID"] = s.ID;     
        row["Study#"] = new List<int>();             
        sessionTable.Rows.Add(row);
    }
});

sessions.ForEach(s => {
    var row = sessionTable.NewRow();
    if(sessionTable.Rows.Find(s.ID) == null){
        row["SessionID"] = s.ID;
        row["Study#"] = new List<int>();      
        sessionTable.Rows.Add(row);
    }
});

sessionTable.Rows.Count

25

#### Step 3.5 Fill studyTable

Here you need to define the filters you are looking for. It can be helpful to add a meaningful description to your session.

In [ ]:
// ================== Define Filters ==================
List<(string description, (string key, object value, Type type)[] filter)> Filters = new List<(string, (string, object, Type)[])>();

Filters.Add(("DGdegrees (2|1|2), HeatFlux 0.2", 
    new (string, object, Type)[] {
        ("DGdegree:Temperature", (object)2, typeof(int)), 
        ("id:HeatFlux", (object)0.2, typeof(double))
    }));
Filters.Add(("DGdegrees (2|1|3), HeatFlux 0.2", 
    new (string, object, Type)[] {
        ("DGdegree:Temperature", (object)3, typeof(int)), 
        ("id:HeatFlux", (object)0.2, typeof(double))
    }));
Filters.Add(("DGdegrees (2|1|2), HeatFlux 0.0", 
    new (string, object, Type)[] {
        ("DGdegree:Temperature", (object)2, typeof(int)), 
        ("id:HeatFlux", (object)0.0, typeof(double))
    }));
Filters.Add(("DGdegrees (2|1|2), HeatFlux 0.0", 
    new (string, object, Type)[] {
        ("DGdegree:Temperature", (object)2, typeof(int)), 
        ("id:HeatFlux", (object)0.02, typeof(double))
    }));
Filters.Add(("DGdegrees (2|1|2), HeatFlux 0.0", 
    new (string, object, Type)[] {
        ("DGdegree:Temperature", (object)2, typeof(int)), 
        ("id:HeatFlux", (object)0.002, typeof(double))
    }));
// ================== Define Filters ==================

In [ ]:
// Collect the Sessions for each study
foreach((string description, (string key, object value, Type type)[] filter) study in Filters){

    var sessionsRefColl = sessionsRef.Select(s => s).ToArray();
    var sessionsColl = sessions.Select(s => s).ToArray();
    foreach(var kvp in study.filter){
        sessionsRefColl = sessionsRefColl.Where(s => Convert.ChangeType(s.KeysAndQueries[kvp.key], kvp.type).Equals(Convert.ChangeType(kvp.value, kvp.type))).ToArray();
        sessionsColl = sessionsColl.Where(s => Convert.ChangeType(s.KeysAndQueries[kvp.key], kvp.type).Equals(Convert.ChangeType(kvp.value, kvp.type))).ToArray();
    }

    int refstudy = -1;
    if(sessionsRefColl.Length != 0){
        var row = studyTable.NewRow();
        row["SessionIDs"] = sessionsRefColl.Select(s => s.ID).ToArray();
        row["Study Description"] = study.description + ", reference";
        row["RefStudy#"] = refstudy;
        refstudy = row.Field<int>("Study#");
        if(((Guid[])row["SessionIDs"]).Count() != 0){
            sessionsRefColl.ForEach(s => sessionTable.Rows.Find(s.ID).Field<List<int>>("Study#").Add(row.Field<int>("Study#")));
            studyTable.Rows.Add(row);
        }
    }
    
    if(sessionsColl.Length != 0){
        var row = studyTable.NewRow();
        row["SessionIDs"] = sessionsColl.Select(s => s.ID).ToArray();
        row["Study Description"] = study.description;
        row["RefStudy#"] = refstudy;
        if(((Guid[])row["SessionIDs"]).Count() != 0){
            sessionsColl.ForEach(s => sessionTable.Rows.Find(s.ID).Field<List<int>>("Study#").Add(row.Field<int>("Study#")));
            studyTable.Rows.Add(row);
        }
    }
    
}
Filters.Clear(); // remove filters, to avoid adding studies again...

#### Step 3.6 - Perform the actual Postprocessing

Now that we have sorted which studies we want to run, we fill the DataTable with the actual Postprocessing data.

In [ ]:
foreach(DataRow row in studyTable.Rows){

    // Plot Data
    var sessions2process = row.Field<Guid[]>("SessionIDs").Select(g => allSessions.Single(s => s.ID == g)).ToArray();    
    row["Contactline Plot Data"] = sessions2process.ReadLogDataForMovingContactLine()[1].ToArray();
    row["Massflux Plot Data"] = sessions2process.ToList().ReadLogDataForXNSE("Massflux").ToArray();

    // Convergence Data
    List<ITimestepInfo> ts = sessions2process.Select(s => s.Timesteps.Newest()).ToList();
    double endTime = ts.First().PhysicalTime;

    // convergence only makes sense if all endtimes are equal in the study
    if(Array.TrueForAll(ts.Select(t => t.PhysicalTime).ToArray(), t => Math.Abs(t-endTime) < 1e-6)){
        List<Plot2Ddata> convPlts= new List<Plot2Ddata>();

        double[] GridRes;
        Dictionary<string, long[]> __DOFs;
        Dictionary<string, double[]> L2Errors;
        Guid[] timestepIds;
        string[] fieldnames = BoSSS.Solution.NSECommon.VariableNames.VelocityVector(2).Cat(BoSSS.Solution.NSECommon.VariableNames.Pressure).Cat(BoSSS.Solution.NSECommon.VariableNames.Temperature).Cat(BoSSS.Solution.NSECommon.VariableNames.LevelSetCG);
        DGFieldComparison.ComputeErrors(fieldnames.ToList(), ts, out GridRes, out __DOFs, out L2Errors, out timestepIds, NormType.L2_embedded); // Extrapolates XDGFields to levelset position on the finest grid, using the Shadowfields
        
        var plt = new Plot2Ddata();
        plt.LogX = true;
        plt.LogY = true;
        foreach(string field in fieldnames){
            int pp;
            if(sessions2process.First().KeysAndQueries.TryGetValue("DGdegree:"+field, out object p)){
                pp = Convert.ToInt32(p);
            } else if(sessions2process.First().KeysAndQueries.TryGetValue("DGdegree:"+field.TrimEnd('X', 'Y', 'Z') + "*", out object ps)) {
                pp = Convert.ToInt32(ps);
            } else {
                pp = 0;
            }
            Dictionary<string, double[][]> dataGroups = new Dictionary<string, double[][]>();
            dataGroups.Add(field +"$k="+pp+"$", new double[2][] { GridRes, L2Errors[field] });
            convPlts.Add(new Plot2Ddata(dataGroups.ToArray()).WithLogX().WithLogY());
        }
        row["Convergence Data"] = convPlts.ToArray();
    }   
}

number of contact lines: 2
Element at 0: time vs contact-pointX
Element at 1: time vs contact-pointY
Element at 2: time vs contact-VelocityX
Element at 3: time vs contact-VelocityY
Element at 4: time vs contact-angle
Element at 0: time vs mass-liq
Element at 1: time vs mass-vap
Element at 2: time vs mass-total
Element at 3: time vs masschange-evap
Element at 4: time vs masschange-vapor
Element at 5: time vs masschange-liquid
Element at 6: time vs masschange-total
Element at 7: time vs interface length
number of contact lines: 2
Element at 0: time vs contact-pointX
Element at 1: time vs contact-pointY
Element at 2: time vs contact-VelocityX
Element at 3: time vs contact-VelocityY
Element at 4: time vs contact-angle
Element at 0: time vs mass-liq
Element at 1: time vs mass-vap
Element at 2: time vs mass-total
Element at 3: time vs masschange-evap
Element at 4: time vs masschange-vapor
Element at 5: time vs masschange-liquid
Element at 6: time vs masschange-total
Element at 7: time vs int

### Step 4 - Serialize Data

In [ ]:
using System.IO;

In [ ]:
using(var tw = new StringWriter()) {
    string Ret = sessionTable.Serialize(); 
    File.WriteAllText(".\\HeatedWall_Validation_SessionTable.json", Ret);    
    Ret = studyTable.Serialize(); 
    File.WriteAllText(".\\HeatedWall_Validation_StudyTable.json", Ret);  
} 

In [ ]:
// deserialize like so:
// DataSet serDataSet = new DataSet();
// using(var tr = new StringReader(File.ReadAllText(".\\HeatedWall_Validation_SessionTable.json"))) {    
//     var obj = TableExtensions.Deserialize(tr.ReadToEnd());
//     obj.TableName = "sessionTable";
//     serDataSet.Tables.Add(obj);
// }
// using(var tr = new StringReader(File.ReadAllText(".\\HeatedWall_Validation_StudyTable.json"))) {    
//     var obj = TableExtensions.Deserialize(tr.ReadToEnd());
//     obj.TableName = "studyTable";
//     serDataSet.Tables.Add(obj);
// }
